# Movie Reviews Classification with DL
adapted from https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/keras/text_classification_with_hub.ipynb#scrollTo=xpKOoWgu-llD

In [1]:
!pip install tensorflow-hub
!pip install tensorflow-datasets

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
import os
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices("GPU") else "NOT AVAILABLE")

Version:  2.8.0
Eager mode:  True
Hub version:  0.12.0
GPU is available


In [3]:
physical_devices = tf.config.list_physical_devices('GPU')
try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
    # Invalid device or cannot modify virtual devices once initialized.
    pass

In [4]:
# Split the training set into 60% and 40% to end up with 15,000 examples
# for training, 10,000 examples for validation and 25,000 examples for testing.
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

In [5]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(1)))
#train_examples_batch

In [6]:
train_labels_batch

<tf.Tensor: shape=(1,), dtype=int64, numpy=array([0])>

In [7]:
embedding = "https://tfhub.dev/google/nnlm-en-dim128-with-normalization/2"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(1, 128), dtype=float32, numpy=
array([[ 1.1501507 ,  0.07801301,  0.09266154,  0.28336143,  0.00967164,
        -0.14918606,  0.33566573, -0.3502448 , -0.00828831, -0.01877141,
        -0.03330698, -0.6330942 , -0.37542126, -0.27773285, -0.09661756,
         0.17255363, -0.13367657,  0.03807659, -0.2751382 ,  0.4947623 ,
         0.03930518,  0.13449623, -0.27072823,  0.01789428, -0.24107152,
         0.02710898,  0.10233342, -0.10662853,  0.05242986,  0.11917047,
        -0.00667077,  0.33923152,  0.11301497,  0.10684258,  0.39157113,
        -0.18953629, -0.17400035, -0.10644415, -0.13420044,  0.17358392,
        -0.27769506, -0.04335914, -0.03915006, -0.1983409 ,  0.02748547,
         0.27670297,  0.14070284, -0.3142561 ,  0.02475693, -0.16730827,
         0.08217402, -0.09568419, -0.25733578,  0.3729148 , -0.22105995,
        -0.2144369 , -0.05832564,  0.15003693, -0.16772385, -0.080281  ,
        -0.21137348, -0.0039831 ,  0.06347676,  0.06651691,  0.0925246 ,
 

In [8]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 128)               124642688 
                                                                 
 dense (Dense)               (None, 16)                2064      
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 124,644,769
Trainable params: 124,644,769
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [10]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=10,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/10
 1/30 [>.............................] - ETA: 44s - loss: 0.7434 - accuracy: 0.5039

ResourceExhaustedError: Graph execution error:

Detected at node 'Adam/Adam/update/ReadVariableOp_2' defined at (most recent call last):
    File "/usr/lib/python3.8/runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/usr/lib/python3.8/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/usr/local/lib/python3.8/dist-packages/ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "/usr/local/lib/python3.8/dist-packages/traitlets/config/application.py", line 846, in launch_instance
      app.start()
    File "/usr/local/lib/python3.8/dist-packages/ipykernel/kernelapp.py", line 505, in start
      self.io_loop.start()
    File "/usr/local/lib/python3.8/dist-packages/tornado/platform/asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "/usr/lib/python3.8/asyncio/base_events.py", line 570, in run_forever
      self._run_once()
    File "/usr/lib/python3.8/asynci

In [ ]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))